# Use CVXPy to solve capital asset pricing model

Here we will seek to solve the capital asset pricing model using CVXPY. To formulate the problem, we will consider Problem 3.3.1 from Shreve which states:

Given an initial wealth $X_{0}$, find an adapted porfolio process $\Delta_{0}, \Delta_{1}...\Delta_{N-1}$ that maximises the utility function
\begin{equation}
\mathbb{E} U(X_{N})\,,
\end{equation}
subject to the wealth equation
\begin{equation}
X_{n+1} = \Delta_{n}S_{n+1} + (1+r)(X_{n} - \Delta_{n}S_{n})
\end{equation}
where $\mathbb{E}$ is the expectation take with respect to the real underlying distribution of the market, and $U(X_{N})$ is the utility function of the final wealth.

As is shown in the text, this problem can be recast as a simpler problem with relation to the state prices of the asses such that we have the following recasting:

Find a vector $\vec{x}$ that maximises $\sum_{n} p_{n}U(x_{n})$ subject to $\sum_{n} p_{n} \zeta_{n} x_{n} = X_{0}$, where $\zeta_{n}$ is the state price given by 
\begin{equation}
    \zeta_{n} = \frac{Z(\omega_{1}\omega_{2}...\omega_{N})}{(1+r)^{N}}\,,
\end{equation}
where $Z = \tilde{\mathbb{P}}/{\mathbb{P}}$ is the Randon-Nikdoynm derivative of the risk neutral probabilty with respect to the actual underlying probability distribution.

Given the symmetries of the Binomial asset pricing model, we can improve efficiency of the calculation by exploiting these symmetries.
This is that the probabilities of different permutations are equal e.g $P(HT) = P(TH)$, which also extends to the risk neutral probabilities, and the the state prices $\zeta$


In [24]:
import cvxpy as cp
import numpy as np
from scipy.special import binom

# Here we will use the functionality of the binomial asset tree
from Binomial_asset_tree import BinomialTreeEuropean, CapitalAssetPricing

In [25]:
def calculate_Sn(S0, u, n):
    # Compute potential price outcomes
    return S0 * (1/u) ** (np.arange(n, -1, -1)) * u ** (np.arange(0, n + 1, 1))

In [26]:
n = 2   # number of periods
u = 2   # up factor
r = 0.25    # risk free rate


# define the risk neutral probabilities
risk_p = np.array([1/4, 2/4, 1/4])

# define actual probabilities
p = np.array([1/9, 4/9, 4/9])

# Define the Randon Nikodym derivative
Z = risk_p/p
zeta = Z/(1+r)**2
pn_zeta = p*zeta
X0 = 4
Sn = calculate_Sn(X0, u, n-1)

params = {'r': r,
         'u': u,
         'd': 1/u,
         'X0': X0,
         'p': p,
         'n': n}

model = CapitalAssetPricing(params)

model.solve()

In [28]:
print(model.x.value)

[ 2.77777912  5.55555311 11.11111464]


In [8]:
x = cp.Variable(n+1)
prob = cp.Problem(cp.Maximize(p@cp.log(x)), [pn_zeta@x==X0])
prob.solve()

1.9458474871352769

In [9]:
x.value

array([ 2.77777912,  5.55555311, 11.11111464])

In [10]:
# Define some system parameters for the model
u = 2
p = 0.5

N = 2
n = 2
K = 7
S0 = 4
r = 0.25

risk_p = (1 + r - 1/u)/(u-1/u)

# Compute real probs
real_probs = (1-p) ** (np.arange(n, -1, -1)) * (p) ** (np.arange(0, n + 1, 1)) *  binom(n, np.arange(n+1))
# real_probs = np.array([1/9, 4/9, 4/9])
print(real_probs)

risk_probs = (risk_p) ** (np.arange(n, -1, -1)) * (1-risk_p) ** (np.arange(0, n + 1, 1)) *  binom(n, np.arange(n+1))
print(risk_probs)
Z = risk_probs/real_probs

zeta = Z/(1+r)**(2)
pn_zeta = real_probs*zeta
Sn = calculate_Sn(S0, u, n)

[0.25 0.5  0.25]
[0.25 0.5  0.25]


In [207]:
x = cp.Variable(n+1)
prob = cp.Problem(cp.Maximize(real_probs@cp.log(x)), [pn_zeta@x==X0])
prob.solve()

1.8325814628517012

In [208]:
x.value

array([6.25001158, 6.24998841, 6.25001158])

array([0.46065971, 3.68527804, 7.37055299])

# 2/

In [15]:
binom(n, np.arange9)

ValueError: invalid number of arguments

NameError: name 'np' is not defined

array([5.        , 5.00480653, 4.99933432, 4.97751727, 4.96158188,
       4.93502256, 4.94399369, 4.93412762, 4.93768569, 4.95132794,
       4.94668564, 4.9421526 , 4.91102246, 4.90938038, 4.9209154 ,
       4.89309448, 4.9158174 , 4.92509224, 4.90407992, 4.86176824,
       4.86948319, 4.87668603, 4.88102422, 4.87834498, 4.89132324,
       4.88483605, 4.86190894, 4.88612766, 4.86509619, 4.8628696 ,
       4.85596139, 4.86908054, 4.89915867, 4.85151166, 4.80962349,
       4.80642829, 4.78961517, 4.81727617, 4.79063766, 4.77056379,
       4.8126751 , 4.83145198, 4.8281015 , 4.81394678, 4.82136431,
       4.81383924, 4.8120316 , 4.79408118, 4.82188087, 4.82283683,
       4.84300547, 4.85672859, 4.82309675, 4.81872302, 4.79577496,
       4.7873958 , 4.78013145, 4.75795555, 4.71634928, 4.70541767,
       4.72938545, 4.71742501, 4.72383335, 4.76618982, 4.76931404,
       4.75824748, 4.78872766, 4.77220869, 4.76456665, 4.72929325,
       4.75972638, 4.77420492, 4.76166894, 4.70354481, 4.66720